In [ ]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Copy annotation and report list files from google drive
!cp "/content/drive/My Drive/Colab Notebooks/new_carina_annotations_7396.csv" "./new_carina_annotations_7396.csv"
!cp "/content/drive/My Drive/Colab Notebooks/cxr-record-list.csv" "./cxr-record-list.csv"

In [ ]:
# Download mimic.zip file that contains 512xN preprocessed images
!wget -O mimic.zip "removed due to privacy requirements of MIMIC-CXR providers"
!unzip -q mimic.zip # Unzip mimic.zip file
!rm mimic.zip #remove zip file

In [ ]:
# Import libraries
import tensorflow as tf
from tensorflow import losses, optimizers
from tensorflow.keras import Input, Model, models, layers, backend
import numpy as np, pandas as pd

In [ ]:
# Find annotated images in npys and save them into the annotated images folder
import glob
import shutil
import numpy as np

# Create directory for annotated images
!mkdir 'carina_annotated_images'

# Search for all image files recursively
imgFiles = glob.glob('./npys/**/*.npy', recursive=True)

# Read the lines in the annotations file
with open("./new_carina_annotations_7396.csv", 'r') as antFile:
  antLines = antFile.readlines()

# Process each annotation
for antLine in antLines:
  # Search for an image
  imgEnd = antLine.find(",")

  # Get image name
  imgName = antLine[:imgEnd]

  # Find and copy the image file in the annotated images folder
  for i, elem in enumerate(imgFiles):
    if imgName in elem:
      shutil.copyfile(imgFiles[i], "./carina_annotated_images/" + imgName + ".npy")
      break

# Zip the carina_annotated_images folder
!zip -q carina_annotated_images_7396.zip ./carina_annotated_images/*.*

# Copy the zip file into google drive
!cp ./carina_annotated_images_7396.zip "/content/drive/My Drive/Colab Notebooks/carina_annotated_images_7396.zip"

In [ ]:
# Once the annotated image zip is saved, no need to download mimic.zip file anymore
# Just unzip the annotated images from google drive
!unzip -q "/content/drive/My Drive/Colab Notebooks/carina_annotated_images_7396.zip"

In [ ]:
# Create the images and labels for training and testing
import glob #file operations
import pickle as pkl #save/load arrays into/from files
import numpy as np #matrix operations
import pdb #python debugging
import h5py

def create_train_test_data(fold_count, fold_index):
  # Initialize variables
  images_train = []
  labels_train = []
  images_test = []
  labels_test = []

  # Search for all image files recursively
  imageFiles = glob.glob('./carina_annotated_images/*.npy', recursive=False)

  # Set counts
  imgCnt = len(imageFiles)
  testCnt = int(imgCnt / fold_count)

  # Read the contents of the annotations file
  with open("./new_carina_annotations_7396.csv", 'r') as annotationsFile:
    annotations = annotationsFile.read()

  # Process each image file
  imgIndex = 0
  for imageFile in imageFiles:
    # Load image into an array
    img = np.load(imageFile)

    # Remove single-dimensional entries from the image array
    img = np.squeeze(img)

    # Copy into an empty 512x512 image
    emptyImg = np.zeros((512,512), np.uint16)
    emptyImg[0:img.shape[0], 0:img.shape[1]] = img

    # Assign the new image back to original
    img = emptyImg

    # Get image ID from image file path
    imgStart = imageFile.rfind("/") + 1
    imgEnd = imageFile.rfind(".npy")
    imgID = imageFile[imgStart:imgEnd]

    # Search for image in annotations
    imgStart = annotations.find(imgID)
    if imgStart != -1:
      imgStart = annotations.find(",", imgStart) + 1

      # Get y coordinate
      yStart = imgStart
      yEnd = annotations.find(",", yStart)
      y = annotations[yStart:yEnd]

      # Get x coordinate
      xStart = yEnd + 1
      xEnd = annotations.find("\n", xStart)
      x = annotations[xStart:xEnd]

      # For each cross validation fold, 20% of images are used for test and 80% for train
      if imgIndex >= testCnt*(fold_index-1) and imgIndex < testCnt*fold_index:
        # Append to test images in flattened form to hold in one row
        images_test.append(img.flatten())

        # Append to test labels
        labels_test.append([y,x])
      else:
        # Append to train images in flattened form to hold in one row
        images_train.append(img.flatten())

        # Append to train labels
        labels_train.append([y,x])

      imgIndex += 1
  
  # Convert lists to np arrays and normalize
  x_train = np.array(images_train).astype(np.float32)
  y_train = np.array(labels_train).astype(np.float32)
  for i in range (len(y_train)):
    x_train[i] = (x_train[i] - np.mean(x_train[i])) / np.std(x_train[i])
    y_train[i] /= 511

  x_test = np.array(images_test).astype(np.float32)
  y_test = np.array(labels_test).astype(np.float32)
  for i in range (len(y_test)):
    x_test[i] = (x_test[i] - np.mean(x_test[i])) / np.std(x_test[i])
    y_test[i] /= 511

  del images_train
  del labels_train
  del images_test
  del labels_test

  # Print training image count
  print("Train Total: " + str(len(y_train)))

  # Print test image count
  print("Test Total: " + str(len(y_test)))

  return x_train, y_train, x_test, y_test

In [ ]:
# Create the images and labels to build a model
import glob #file operations
import pickle as pkl #save/load arrays into/from files
import numpy as np #matrix operations
import pdb #python debugging
import h5py

def create_train_data():
  # Initialize variables
  images_train = []
  labels_train = []

  # Search for all image files recursively
  imageFiles = glob.glob('./carina_annotated_images/*.npy', recursive=False)

  # Read the contents of the annotations file
  with open("./new_carina_annotations_7396.csv", 'r') as annotationsFile:
    annotations = annotationsFile.read()

  # Process each image file
  imgIndex = 0
  for imageFile in imageFiles:
    # Load image into an array
    img = np.load(imageFile)

    # Remove single-dimensional entries from the image array
    img = np.squeeze(img)

    # Copy into an empty 512x512 image
    emptyImg = np.zeros((512,512), np.uint16)
    emptyImg[0:img.shape[0], 0:img.shape[1]] = img

    # Assign the new image back to original
    img = emptyImg

    # Get image ID from image file path
    imgStart = imageFile.rfind("/") + 1
    imgEnd = imageFile.rfind(".npy")
    imgID = imageFile[imgStart:imgEnd]

    # Search for image in annotations
    imgStart = annotations.find(imgID)
    if imgStart != -1:
      imgStart = annotations.find(",", imgStart) + 1

      # Get y coordinate
      yStart = imgStart
      yEnd = annotations.find(",", yStart)
      y = annotations[yStart:yEnd]

      # Get x coordinate
      xStart = yEnd + 1
      xEnd = annotations.find("\n", xStart)
      x = annotations[xStart:xEnd]

      # Append to train images in flattened form to hold in one row
      images_train.append(img.flatten())

      # Append to train labels
      labels_train.append([y,x])

      imgIndex += 1
  
  # Convert lists to np arrays and normalize
  x_train = np.array(images_train).astype(np.float32)
  y_train = np.array(labels_train).astype(np.float32)

  # Comment out section below for augmentation
  for i in range (len(y_train)):
    x_train[i] = (x_train[i] - np.mean(x_train[i])) / np.std(x_train[i])
    y_train[i] /= 511

  del images_train
  del labels_train

  # Print training image count
  print("Train Total: " + str(len(y_train)))

  return x_train, y_train

In [ ]:
# Show a random set of images and their labels
import numpy as np
import pylab as py

# Create train and test data for the first cross validation fold
x_train, y_train, x_test, y_test = create_train_test_data(5, 1)

# Print 16 random images
fig = py.figure()
for i in range(16):
  imIndex = np.random.randint(x_train.shape[0])
  im = x_train[imIndex].reshape(512, 512)
  fig.add_subplot(4, 4, i + 1)
  py.imshow(im)
  py.axis('off')
  print(y_train[imIndex])

In [ ]:
# Prepare model method
def prepare_model(inputs):
  # Define kwargs dictionary
  kwargs = {
      'kernel_size': (3, 3),
      'padding': 'same'}

  # Define lambda functions
  conv = lambda x, filters, strides : layers.Conv2D(filters=filters, strides=strides, **kwargs)(x)
  norm = lambda x : layers.BatchNormalization()(x)
  relu = lambda x : layers.LeakyReLU()(x)

  # Define stride-1, stride-2 blocks
  conv1 = lambda filters, x : relu(norm(conv(x, filters, strides=1)))
  conv2 = lambda filters, x : relu(norm(conv(x, filters, strides=(2, 2))))

  # Define contracting layers
  l1 = conv2(48, conv1(48, conv1(48, conv1(48, conv1(48, conv1(48, inputs['dat']))))))
  l2 = conv2(56, conv1(56, conv1(56, conv1(56, conv1(56, l1)))))
  l3 = conv2(64, conv1(64, conv1(64, conv1(64, conv1(64, l2)))))
  l4 = conv2(80, conv1(80, conv1(80, conv1(80, l3))))
  l5 = conv2(96, conv1(96, conv1(96, conv1(96, l4))))
  l6 = conv2(112, conv1(112, conv1(112, l5)))
  l7 = conv2(128, conv1(128, conv1(128, l6)))

  # Fully connected layer
  fc = layers.Dense(128, activation="relu", name="fc")
  l8 = fc(layers.Flatten()(l7))

  # Logit layer
  c1 = layers.Reshape((1, 1, 1, 128))(l8)
  c2 = layers.Conv3D(filters=2, kernel_size=(1, 1, 1), activation='sigmoid')(c1)

  # Create logits
  logits = {}
  logits['car'] = layers.Reshape((-1, 2), name='car')(c2)
  
  # Create model
  model = Model(inputs=inputs, outputs=logits) 

  return model

In [ ]:
# Test model method
import math

def test_model(fold, x_test, y_test):
  distInPixelsTest = 0
  distInCmTest = 0
  totalCnt = len(y_test)
  for i in range(totalCnt):
    # Predict normalized values of carina coordinates
    logits = model.predict(x=x_test[i].reshape(1, 512, 512, 1))

    # Denormalize predictions back to coordinates
    predY = logits.flatten()[0]*511
    predX = logits.flatten()[1]*511
    labelY = y_test[i][0]*511
    labelX = y_test[i][1]*511
    distInPixels = math.sqrt((predY - labelY) ** 2 + (predX - labelX) ** 2)
    distInCm = distInPixels * 0.08
    distInPixelsTest += distInPixels
    distInCmTest += distInCm
    with open('carina_512_distances.csv', 'a') as carDistFile:
      carDistFile.write(str(fold) + ',' + str(distInPixels) + '\n')

  # Calculate test results
  distInPixelsAvg = distInPixelsTest/totalCnt
  distInCmAvg = distInCmTest/totalCnt

  print('Avg Dist in Pixels: ', distInPixelsAvg)
  print('Avg Dist in Cm: ', distInCmAvg)

In [ ]:
#--------------------------------------------
# Train and test with 5-fold cross validation
#--------------------------------------------

import datetime

with open('carina_512_distances.csv', 'a') as carDistFile:
  carDistFile.write('fold,dist\n')

for fold in range(5):
  print('fold: ' + str(fold))

  # Create model inputs
  inputs = {}
  inputs['dat'] = Input(shape=(512, 512, 1))

  # Prepare the model
  model = prepare_model(inputs)

  # Create train and test data for the cross validation fold
  x_train, y_train, x_test, y_test = create_train_test_data(5, fold + 1)

  # Initialize learning rate and epoch
  lr = 0.0005
  epoch = 1

  for j in range(5):
    # Compile the model
    model.compile(
        optimizer=optimizers.Adam(learning_rate=lr),
        loss={'car': losses.MeanSquaredError()})

    for i in range(2):
      # Print learning-rate and epoch
      print('learning-rate: ' + str(lr))
      print('epoch: ' + str(epoch))

      # Train the model
      model.fit(
          x=x_train.reshape(x_train.shape[0], 512, 512, 1), 
          y=y_train,
          batch_size=1,
          steps_per_epoch=None, 
          epochs=1)

      # Increment epoch
      epoch += 1

      # Check epoch
      if epoch == 10:
        break

    # Check epoch
    if epoch == 10:
      break
    else:
      lr /= 2

  # Set start time
  startTime = datetime.datetime.now().time()

  # Test the model
  test_model(fold, x_test, y_test)

  # Set end time
  endTime = datetime.datetime.now().time()

  # Print start and end times
  print("Start Time: " + str(startTime))
  print("End Time: " + str(endTime))
  print("Item Count: " + str(len(y_test)))

  # Delete the model at the end of each fold
  del model
  del x_train
  del y_train
  del x_test
  del y_test

# Copy the error distance file to google file
!cp ./carina_512_distances.csv '/content/drive/My Drive/Colab Notebooks/carina_512_7396_distances.csv'

In [ ]:
#--------------------------------------------------
# Generate a model file from the whole training set
#--------------------------------------------------

# Create model inputs
inputs = {}
inputs['dat'] = Input(shape=(512, 512, 1))

# Prepare the model
model = prepare_model(inputs)

# Create train data to build a model
x_train, y_train = create_train_data()

# Initialize learning rate and epoch
lr = 0.0005
epoch = 1

for j in range(5):
  # Compile the model
  model.compile(
      optimizer=optimizers.Adam(learning_rate=lr),
      loss={'car': losses.MeanSquaredError()})

  for i in range(2):
    # Print learning-rate and epoch
    print('learning-rate: ' + str(lr))
    print('epoch: ' + str(epoch))

    # Train the model
    model.fit(
        x=x_train.reshape(x_train.shape[0], 512, 512, 1), 
        y=y_train,
        batch_size=1,
        steps_per_epoch=None, 
        epochs=1)

    # Increment epoch
    epoch += 1

    # Check epoch
    if epoch == 10:
      break

  # Check epoch
  if epoch == 10:
    break
  else:
    lr /= 2

In [ ]:
# --- Save the model to a file
model.save('./cnn_1_carina_regression.hdf5')

# Copy the model to google drive
!cp ./cnn_1_carina_regression.hdf5 '/content/drive/My Drive/Colab Notebooks/cnn_1_carina_regression.hdf5'

In [ ]:
# Copy the model file from google drive
!cp '/content/drive/My Drive/Colab Notebooks/cnn_1_carina_regression.hdf5' ./cnn_1_carina_regression.hdf5

# Load the model from the file
from tensorflow.keras import models as tfModels
model = tfModels.load_model('./cnn_1_carina_regression.hdf5', compile=False)

In [ ]:
def crop_image(imgS, hS, wS, yC, xC, hT, wT):
  # Carina must be at (.5, .75) in terms of (x, y)
  cL = wT // 2
  cR = cL
  cT = hT * 3 // 4
  cB = hT - cT

  # Calculate left and right x coordinates, top and bottom y coordinates of
  # the 256x128 cropped area around carina on the original 512x512 image
  xL = xC - cL
  xR = xC + cR
  yT = yC - cT
  yB = yC + cB

  # Calculate width by checking 3 possible cases of xL and xR
  if xL < 0:
    w = wT - abs(xL)
  elif xR > (wS - 1):
    w = wT - abs(xR - (wS - 1))
  else:
    w = wT

  # Calculate height by checking 3 possible cases of yT and yB 
  if yT < 0:
    h = hT - abs(yT)
  elif yB > (hS - 1):
    h = hT - abs(yB - (hS - 1))
  else:
    h = hT

  # Calculate x0 on original  
  if xL < 0:
    x0 = 0
  else:
    x0 = xL

  # Calculate y0 on original  
  if yT < 0:
    y0 = 0
  else:
    y0 = yT

  # Calculate x1 on target  
  if xL < 0:
    x1 = abs(xL)
  else:
    x1 = 0

  # Calculate y1 on target  
  if yT < 0:
    y1 = abs(yT)
  else:
    y1 = 0

  # Create an empty target image
  imgT = np.zeros((hT, wT), np.uint16)

  # Copy the cropped part of the image from source to target
  np.copyto(imgT[y1:, x1:], imgS[y0:y0+h, x0:x0+w])

  return imgT

In [ ]:
import tensorflow as tf
import os
import glob
import numpy as np

# Use the model to find the carina in all images under npys, and
# crop an area of 256x128 around the carina to have it located at [.75, .5]
def crop_images_around_carina(hS, wS, hT, wT):
  # Read the contents of the frontal file
  with open("./db-mimic-frontal-only.csv", 'r') as frontalFile:
    frontalContent = frontalFile.read()

  # Read the contents of the annotations file
  with open("./new_carina_annotations_7396.csv", 'r') as annotationsFile:
    annotations = annotationsFile.read()

  # Search for all image files recursively
  imgFiles = glob.glob('./npys/**/*.npy', recursive=True)

  # Process each image file
  for imgFile in imgFiles:
    # Get image name
    imgStart = imgFile.rfind("/") + 1
    imgEnd = imgFile.rfind(".npy")
    imgName = imgFile[imgStart:imgEnd]

    # Skip the image if it's not frontal
    if frontalContent.find(imgName) == -1:
      continue

    # Load an original image such as 512xN or Nx512
    imgO = np.load(imgFile)

    # Create an empty source image
    imgS = np.zeros((hS, wS), np.uint16)

    # Insert the original image to the top left corner of the source image
    imgS[0:imgO.shape[0], 0:imgO.shape[1]] = imgO
    
    # Initialize x and y coordinates
    x = wS / 2
    y = hS * 3 / 4

    # Search for image in annotations
    antStart = annotations.find(imgName)
    if antStart != -1:
      # Get y coordinate
      yStart = annotations.find(",", antStart) + 1
      yEnd = annotations.find(",", yStart)
      y = int(float(annotations[yStart:yEnd]))

      # Get x coordinate
      xStart = yEnd + 1
      xEnd = annotations.find("\n", xStart)
      x = int(float(annotations[xStart:xEnd]))
    else:
      # Normalize the source image
      imgN = (imgS - np.mean(imgS)) / np.std(imgS)

      # Predict coordinates of carina
      logits = model.predict(imgN.reshape(1, hS, wS, 1))

      # Denormalize predictions back to coordinates
      y = int(logits.flatten()[0]*511)
      x = int(logits.flatten()[1]*511)

    # Crop image around carina
    croppedImg = crop_image(imgS, hS, wS, y, x, hT, wT)

    # Save image at its new directory
    croppedImgFile = imgFile.replace("npys", "crops")
    directory = os.path.dirname(croppedImgFile)
    if not os.path.exists(directory):
      os.makedirs(directory)
    np.save(croppedImgFile, croppedImg)

# Crop images around carina
crop_images_around_carina(512, 512, 256, 128)

# Zip the crops folder
!zip -qr crops.zip ./crops

# Cop the zip file to google drive
!cp crops.zip "/content/drive/My Drive/Colab Notebooks/crops.zip"

# Print the number of cropped images
print(len(glob.glob('./crops/**/*.npy', recursive=True)))